In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
# load data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# flatten 28*28 images to a 784 vector for each image
num_pixels = X_train.shape[1] * X_train.shape[2]
X_train = X_train.reshape((X_train.shape[0], num_pixels)).astype('float32')
X_test = X_test.reshape((X_test.shape[0], num_pixels)).astype('float32')
# normalize inputs from 0-255 to 0-1
X_train = X_train / 255
X_test = X_test / 255
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]
# define baseline model
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(num_pixels, input_dim=num_pixels, kernel_initializer='normal', activation='relu'))
	model.add(Dense(num_classes, kernel_initializer='normal', activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# build the model
model = baseline_model()
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200, verbose=2)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

Epoch 1/10
300/300 - 2s - loss: 0.2783 - accuracy: 0.9218 - val_loss: 0.1395 - val_accuracy: 0.9588 - 2s/epoch - 8ms/step
Epoch 2/10
300/300 - 2s - loss: 0.1090 - accuracy: 0.9690 - val_loss: 0.0935 - val_accuracy: 0.9721 - 2s/epoch - 5ms/step
Epoch 3/10
300/300 - 2s - loss: 0.0719 - accuracy: 0.9790 - val_loss: 0.0763 - val_accuracy: 0.9771 - 2s/epoch - 5ms/step
Epoch 4/10
300/300 - 2s - loss: 0.0511 - accuracy: 0.9854 - val_loss: 0.0645 - val_accuracy: 0.9801 - 2s/epoch - 5ms/step
Epoch 5/10
300/300 - 2s - loss: 0.0376 - accuracy: 0.9888 - val_loss: 0.0631 - val_accuracy: 0.9800 - 2s/epoch - 6ms/step
Epoch 6/10
300/300 - 2s - loss: 0.0262 - accuracy: 0.9926 - val_loss: 0.0678 - val_accuracy: 0.9793 - 2s/epoch - 6ms/step
Epoch 7/10
300/300 - 2s - loss: 0.0196 - accuracy: 0.9951 - val_loss: 0.0581 - val_accuracy: 0.9817 - 2s/epoch - 6ms/step
Epoch 8/10
300/300 - 2s - loss: 0.0145 - accuracy: 0.9967 - val_loss: 0.0620 - val_accuracy: 0.9810 - 2s/epoch - 6ms/step
Epoch 9/10
300/300 - 2s 

In [2]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.060268133878707886
Test accuracy: 0.9815999865531921


In [6]:
from keras.models import load_model
from tkinter import *
import tkinter as tk
import win32gui
from PIL import ImageGrab, ImageOps
import numpy as np

model = load_model('mnist.h5')

def predict_digit(img):
#resize image to 28×28 pixels
        img = img.resize((28,28))
        #convert rgb to grayscale
        img = img.convert('L')
        img = ImageOps.invert(img)
        img = np.array(img)
        #reshaping to support our model input and normalizing
        img = img.reshape(1,28,28,1)
        img = img/255.0
        #predicting the class
        res = model.predict([img])[0]
        return np.argmax(res), max(res)

class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)

        self.x = self.y = 0

        # Creating elements
        self.canvas = tk.Canvas(self, width=300, height=300, bg = "white", cursor="cross")
        self.label = tk.Label(self, text="Thinking..", font=("Helvetica", 48))
        self.classify_btn = tk.Button(self, text = "Recognise", command =         self.classify_handwriting) 
        self.button_clear = tk.Button(self, text = "Clear", command = self.clear_all)

        # Grid structure
        self.canvas.grid(row=0, column=0, pady=2, sticky=W, )
        self.label.grid(row=0, column=1,pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)

        #self.canvas.bind("<Motion>", self.start_pos)
        self.canvas.bind("<B1-Motion>", self.draw_lines)

    def clear_all(self):
        self.canvas.delete("all")

    def classify_handwriting(self):
        HWND = self.canvas.winfo_id() # get the handle of the canvas
        rect = win32gui.GetWindowRect(HWND) # get the coordinate of the canvas
        im = ImageGrab.grab(rect)

        digit, acc = predict_digit(im)
        self.label.configure(text= str(digit)+', '+ str(int(acc*100))+'%')

    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r=8
        self.canvas.create_oval(self.x-r, self.y-r, self.x + r, self.y + r, fill='black')

app = App()
mainloop()